# Imports des librairies

In [36]:
import os 
from PIL import Image
import matplotlib.pyplot as plt

# Exploration du jeu de données 

## Import de la base

*Pour récupérer le Dataset, on a automatisé ça en récupérant le token de l'API Kaggle*  

*Pour récupérer le Dataset :*  
- Aller sur [Kaggle](https://www.kaggle.com/).  
- Se connecter à son compte ou s'inscrire.  
- Cliquer en haut à droite sur l'icône de son profil.  
- Aller dans la rubrique API de l'onglet settings puis "create new token".    

*Le token de l'API Kaggle est maintenant téléchargé.*  

In [ ]:
import os
import json

kaggle_path = os.path.expanduser("~/.kaggle")
kaggle_json_path = os.path.expanduser("~/Downloads/kaggle.json")

with open(kaggle_json_path, "r") as f:
    kaggle_token = json.load(f)

with open(os.path.join(kaggle_path, "kaggle.json"), "w") as f:
    json.dump(kaggle_token, f)

os.chmod(os.path.join(kaggle_path, "kaggle.json"), 0o600)

In [ ]:
!kaggle datasets download -d mikoajkolman/pokemon-images-first-generation17000-files --unzip -p "/Users/gabriel/pokemon_nn/images_pokemon"

Dataset URL: https://www.kaggle.com/datasets/mikoajkolman/pokemon-images-first-generation17000-files
License(s): GPL-2.0
100%|██████████████████████████████████████| 2.81G/2.81G [04:31<00:00, 10.2MB/s]
100%|██████████████████████████████████████| 2.81G/2.81G [04:31<00:00, 11.1MB/s]


*Le jeu de données est déjà très propre nous n'avons pas à le clean*

## Preview des images 

*Avant de visualiser les images, on voit qu'elles ne sont pas toutes dans le meme format.  
On va donc standardiser tous ces formats pour avoir des images qui sont toutes au même format.*

In [ ]:
def les_extensions_dossier_ss_dossier (dossier_parent):
    """ 
    Fonction qui parcourt un dossier et ses sous-dossiers pour en extraire les extensions de fichiers.
    
    Paramètres:
    - dossier_parent (str) : chemin du dossier parent à parcourir.

    Retourne:
    - extensions (set) : ensemble des extensions de fichiers trouvées dans le dossier et ses sous-dossiers.
    
    Exemple d'utilisation:
    extensions = les_extensions_dossier_ss_dossier("/Users/gabriel/Desktop/pokemon")
    print(extensions)
    """
    extensions = set()
    for root, sous_dossier, fichiers in os.walk(dossier_parent):
        for fichier in fichiers:
            ext = os.path.splitext(fichier)[1].lower()  
            extensions.add(ext)
    return extensions

dossier_parent = "/Users/gabriel/Desktop/pokemon"
extensions = les_extensions_dossier_ss_dossier(dossier_parent)
extensions


{'.bmp', '.gif', '.jpeg', '.jpg', '.png'}

In [91]:
def convertir_en_png(dossier_parent, extensions={'.jpg', '.jpeg', '.gif', '.bmp'}):
    """ 
    Convertit toutes les images dans un dossier et ses sous-dossiers en PNG.
    Remplace les anciennes versions par la nouvelle en écrasant les fichiers.
    Renomme également les fichiers en `.png` si l'extension est mal écrite.

    Paramètres:
    - dossier_parent (str) : chemin du dossier parent à parcourir.
    - extensions (set) : ensemble des extensions de fichiers à convertir. 

    Exemple d'utilisation:
    convertir_en_png("/Users/gabriel/Desktop/pokemon")
    """

    for root, sous_dossier, fichiers in os.walk(dossier_parent):
        for fichier in fichiers:
            file_path = os.path.join(root, fichier)
            extension = os.path.splitext(fichier)[1].lower()

            if extension in extensions:
                try:
                    with Image.open(file_path) as img:
                        new_file_path = os.path.splitext(file_path)[0] + ".png"
                        img.convert("RGBA").save(new_file_path, format="PNG")
                        os.remove(file_path)
                except Exception as e:
                    print(f"Erreur avec {file_path} : {e}")

            elif extension.startswith(".png") and extension != ".png":
                new_file_path = os.path.splitext(file_path)[0] + ".png"
                os.rename(file_path, new_file_path)
                print(f"Renommé : {file_path} → {new_file_path}")

dossier_parent = "/Users/gabriel/Desktop/pokemon"
convertir_en_png(dossier_parent)

In [ ]:
def affiche_pokemon (dossier_parent, nom_pokemon, numero_image ) : 
    """
    Fonction qui affiche une image de Pokémon à partir de son nom et de son numéro d'image.
    
    Paramètres:
    - dossier_parent (str) : chemin du dossier parent contenant les images.
    - nom_pokemon (str) : nom du Pokémon.
    - numero_image (int) : numéro de l'image du Pokémon.
    """
    chemin_image = os.path.join(dossier_parent, nom_pokemon, f"{numero_image}.png")
    img = plt.imread(chemin_image)
    plt.imshow(img)
    plt.show()


In [58]:
def downscale_image (dossier_parent, nom_pokemon, numero_image, new_size=(64, 64), sauvegarde = False):
    """
    Fonction qui réduit la taille d'une image de Pokémon à une taille spécifiée.
    
    Paramètres:
    - dossier_parent (str) : chemin du dossier parent contenant les images.
    - nom_pokemon (str) : nom du Pokémon.
    - numero_image (int) : numéro de l'image du Pokémon.
    - new_size (tuple) : nouvelle taille de l'image (largeur, hauteur).
        Par défaut : (64, 64).
    
    """
    chemin_image = os.path.join(dossier_parent, nom_pokemon, f"{numero_image}.png")
    img = Image.open(chemin_image)
    img_resized = img.resize(new_size)
    return img_resized

In [ ]:
def reindex_image (dossier_parent, nom_pokemon):
    """
    Fonction qui renomme une image d'un Pokemon pour que les index se suivent.
    
    Paramètres:
    - dossier_parent (str) : chemin du dossier parent contenant les images.
    - nom_pokemon (str) : nom du Pokémon cible.
    
    """
    chemin_dossier = os.path.join(dossier_parent, nom_pokemon)
    fichiers = os.listdir(chemin_dossier)
    fichiers_png = [f for f in fichiers if f.lower().endswith('.png')]
    
    for i, fichier in enumerate(fichiers_png):
        ancien_chemin = os.path.join(chemin_dossier, fichier)
        nouveau_nom = f"{nom_pokemon+str(i)}.png"
        nouveau_chemin = os.path.join(chemin_dossier, nouveau_nom)
        os.rename(ancien_chemin, nouveau_chemin)
    return None

In [84]:
def rename_tout_reindex(dossier_parent):
    """
    Fonction qui renomme toutes les images de Pokémon dans un dossier pour que les index se suivent.
    
    Paramètres:
    - dossier_parent (str) : chemin du dossier parent contenant les images.
    
    """
    for nom_pokemon in os.listdir(dossier_parent):
        chemin_dossier = os.path.join(dossier_parent, nom_pokemon)
        if os.path.isdir(chemin_dossier):  
            reindex_image(dossier_parent, nom_pokemon)

    return None